In [ ]:
from surprise import SVD, NMF, Dataset, Reader, SVDpp, BaselineOnly, KNNBaseline, SlopeOne, accuracy
from surprise.model_selection import cross_validate, GridSearchCV,train_test_split, KFold, GridSearchCV
from sklearn.model_selection import KFold as skFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
import pandas as pd 
from project_helpers import *
from math import *

#seed
random.seed(404)
np.random.seed(404)

In [ ]:
train = pd.read_csv(r'data_train.csv')
train = df_to_surprise(train)

#Split data into models training_set and blender_set
traing_set = train.sample(frac = 0.8, random_state = 200)
blender_set = train.drop(traing_set.index)
mean = global_mean(traing_set)
users = user_mean(traing_set)
movies = movie_mean(traing_set)

#Change both dataset into the surprise format
#setup the rating scale
reader = Reader(rating_scale=(1, 5))

traing_set_surp = Dataset.load_from_df(traing_set, reader)
traing_set_surp_train = traing_set_surp.build_full_trainset()
blend_surp = Dataset.load_from_df(blender_set, reader)
blend_surp_train = blend_surp.build_full_trainset()

#Load blend train set as a testset for models performance evaluation
blend_surp_test = blend_surp_train.build_testset()

In [ ]:
#Gread search
#GS for Basline 
grid_baseline = {'bsl_options': {'method': ['sgd'],'reg': [10**-i for i in range(-3,12)]},'verbose':[False]}
gs_baseline = GridSearchCV(BaselineOnly, grid_baseline, measures = ['rmse'], cv = KFold(n_splits = 3, random_state = 200, shuffle = False),n_jobs = 8)
#n_jobs = 8 means we use 8CPU 
gs_baseline.fit(traing_set_surp)
print('Best Hyperparameters Basline : ', gs_baseline.best_params['rmse'])

#GS for SVD + baseline
grid_SVDb = {'reg_all': [10**-i for i in range(-1,2)], 'biased':[True], 'n_factors':[300, 400],'n_epochs':[500], 'lr_all':[0.0015]}
gs_SVDb = GridSearchCV(SVD, grid_SVDb, measures = ['rmse'], cv = KFold(n_splits = 3, random_state = 200, shuffle = False),n_jobs = 8)
gs_SVDb.fit(traing_set_surp)
print('Best Hyperparameters SVD + baseline: ', gs_SVDb.best_params['rmse'])

#SVD without baseline
grid_SVD = {'reg_all':[10**-i for i in range(-3,12)], 'biased':[False], 'n_factors':[1, 5, 10, 100, 200],'n_epochs':[500], 'lr_all':[0.0015]}
gs_SVD = GridSearchCV(SVD, grid_SVD, measures = ['rmse'], cv = KFold(n_splits = 3, random_state = 200, shuffle = False),n_jobs = 8)
gs_SVD.fit(traing_set_surp)
print('Best Hyperparameters SVD withour baseline: ', gs_SVD.best_params['rmse'])

#GS for KNN for user
grid_knn_user = {'bsl_options': {'method':['als'],'n_epochs':[50]},'k': [150, 200, 250, 300,350,400,450,500,550],
                 'sim_options': {'name': ['pearson_baseline'],'min_support': [1],'user_based': [True]}}
gs_knn_user = GridSearchCV(KNNBaseline, grid_knn_user, measures=['rmse'], cv = KFold(n_splits = 3, random_state = 200
                                                                                     , shuffle = False),n_jobs = 8)
gs_knn_user.fit(traing_set_surp)
print('Best Hyperparameters KNN for user: ', gs_knn_user.best_params['rmse'])

#GS KNN for movie
grid_knn_movie = {'bsl_options': {'method':['als'],'n_epochs':[50]},'k': [150, 200, 250, 300,350,400,450,500,550],
                  'sim_options': {'name': ['pearson_baseline'],'min_support': [1],'user_based': [False]}}
gs_knn_movie = GridSearchCV(KNNBaseline, grid_knn_movie, measures = ['rmse'], cv = KFold(n_splits = 3
                            , random_state = 200, shuffle = False),n_jobs = 8)
gs_knn_movie.fit(traing_set_surp)
print('Best Hyperparameters KNN for movie: ', gs_knn_movie.best_params['rmse'])

#GS NMF
grid_NMF = {'reg_pu': [0.06, 0.1]
            ,'reg_qi': [0.06, 0.1]
            ,'reg_bu': [0.02, 0.2]
            ,'reg_bi': [0.02, 0.2]
            ,'lr_bu': [0.005, 0.001,]
            ,'lr_bi': [0.005, 0.001]
            ,'biased':[True]
            ,'n_factors':[15, 200, 400]
            ,'n_epochs':[50,100,400]}
gs_NMF = GridSearchCV(NMF, grid_NMF, measures=['rmse'], 
                           cv = KFold(n_splits = 3, random_state = 200
                                      , shuffle = False),n_jobs = 8)
gs_NMF.fit(traing_set_surp)
print('Best Hyperparameters NMF: ', gs_NMF.best_params['rmse'])